# Part 3: Data Analysis (40%)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Crawl a real-world dataset

## Step 2: Performa data preparation & cleaning

## Step 3: Perform exploratory analysis and ask questions

## Step 5: Summarise and write a conclusion using markdown cells